# Import libraries

In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import re
import scipy.io
import matplotlib.pyplot as plt
from os.path import join
from glob import glob
from common.hc3_preprocess import crns_get, get_sess_dict, read_whl, get_sr_from_xml, process_position_df
from common.preprocess import get_occupancy, placefield, get_occupancy_bins, placefield_bins, spatial_information, complete_contourline
from common.utils import load_pickle
from common.comput_utils import pair_diff, find_pair_times
from scipy.interpolate import interp1d
from scipy.signal import filtfilt, butter, hilbert
from scipy.io import loadmat
from skimage.measure import find_contours
from skimage.draw import polygon2mask

# Analysis
from common.comput_utils import IndataProcessor, check_border


data_dir = 'data/crcns-hc-3/'
os.makedirs(data_dir, exist_ok=True)
meta_dir = os.path.join(data_dir, 'crcns-hc3-metadata-tables')
related_tasks = ['bigSquare', 'midSquare']
download_dir = os.path.join(data_dir, 'data_zips-tables')
os.makedirs(download_dir, exist_ok=True)
session_df = pd.read_csv(os.path.join(meta_dir, 'hc3-session.csv'),
                         names=['id', 'topdir', 'session', 'behavior', 'fam', 'duration']
                         )
cell_df = pd.read_csv(os.path.join(meta_dir, 'hc3-cell.csv'), 
                      names= ['matlab_id', 'topdir', 'animal', 'ele', 'clu', 'region', 'ne', 'ni', 'eg', 'ig', 'ed', 'id', 'fireRate', 'totalFireRate', 'type', 'eDist', 'RefracRatio', 'RefracViol']
                      )

# Download

Get data from CRCNS website. Downloaded data stored at data/crcns-hc-3

Criteria: (1) behavioral tasks (free foraging), (2) pyramidal cells, (3) CA1 / CA3 regions

In [2]:
delete_list = ['.*fet.*', '.*m1m2.*', '.*m1v', '.*mpg', '.*mpeg', '.*mm.*', '.*spk.*', '.*threshold.*', '.*tar.gz']

# selection based on the criteria (CA1/3, pyramidal, free foraging)
selected_topdir_df = cell_df[(cell_df.type == 'p') & ((cell_df.region == 'CA1') | (cell_df.region == 'CA3'))]
selected_topdirs = selected_topdir_df.topdir.unique()

task_mask = session_df.behavior.isin(related_tasks)
topdir_mask = session_df.topdir.isin(selected_topdirs)
data_to_download = session_df[topdir_mask & task_mask & fam_mask][['topdir', 'session']].reset_index(drop=True)



In [ ]:
# Download data from crnrs website
username, password = None, None
for i in range(data_to_download.shape[0]):
    print('%d/%d' % (i, data_to_download.shape[0]))
    topdir_each, session_each = data_to_download.iloc[i][['topdir', 'session']]
    dataset_identifier = 'hc-3/%s/%s.tar.gz' % (topdir_each, session_each)
    
    if username is None:
        raise ValueError("An account is required. You can register it in the CRNS website")
    filename = crns_get(dataset_identifier,
                         username='',  # An account is required. You can register it in the CRNS website
                         password='',  #
                         dest_dir=download_dir)
    print('Downloaded %s'%(filename))

In [ ]:


# Extract downloaded data
all_zips = glob(os.path.join(download_dir, '*.tar.gz'))
for idx, zip in enumerate(all_zips):
    print('\rExtracting %d/%d: %s' % (idx, len(all_zips), zip))
    shutil.unpack_archive(zip, data_dir)

# Delete irrelevant data to save space (use with cautions)
for x in os.walk(data_dir):
    # print('Dir path = \n', x[0])
    # print('File names = \n', x[2])
    if len(x[2]) > 0:
        for fn in x[2]:
            for each_patten in delete_list:
                match_obj = re.search(each_patten, fn)
                if match_obj:
                    file_to_rm_path = os.path.join(x[0], fn)
                    os.remove(file_to_rm_path)
                    print('Delete: ', file_to_rm_path)
    

In [ ]:
# Run the matlab script to convert the LFP data in .egg format to .mat
# The script smaples 30 channels (seeded) for their lfp, and save them as .mat
# /home/yiu/projects/hippocampus/matlab/hc3_preprocessing.m

# Organize into dataframe

Organize and SAVE all data into a dataframe

Saved dataframe is located at data/dataframes/hc3data_df.pickle

In [2]:
data_dir = 'data/crcns-hc-3/'
position_fs = 39.06  # See dataset description from crcns
excluded_dirs = ['data_zips-tables', 'crcns-hc3-metadata-tables']
sess2topdir, sess2animal, sess2behavior, sess2duration = get_sess_dict(cell_df, session_df)

df_dict = dict(
    animal=[],
    topdir=[],
    session=[],
    duration=[],
    behavior=[],
    position=[],
    CA1_spdf=[],
    CA3_spdf=[],
    wave=[],
    spike_samplingrate=[],
    lfp_samplingrate=[],
    position_samplingrate=[]
)
progress_idx = 0

# Loop topdir
for topdir_each in os.listdir(data_dir):
    if topdir_each in excluded_dirs:
        continue
    nextdir = os.path.join(data_dir, topdir_each)
    if os.path.isdir(nextdir) is False:
        print('Skip', nextdir)
        continue

    # Loop session
    for sess_folder in os.listdir(os.path.join(data_dir, topdir_each)):
        progress_idx += 1
        print('\r%d)' % progress_idx, end='', flush=True)
        sess_dir = os.path.join(data_dir, topdir_each, sess_folder)

        # Rat's position and LFP
        position_file_path = os.path.join(os.path.join(sess_dir, '%s.whl' % sess_folder))
        lfp_path = os.path.join(os.path.join(sess_dir, '%s.mat' % sess_folder))
        if (os.path.isfile(position_file_path)) and (os.path.isfile(lfp_path)):
            position_df = read_whl(position_file_path)
            lfp_allch = loadmat(lfp_path)['sesslfp']
        else:
            print('%s/%s .whl or . mat not found' % (topdir_each, sess_folder))
            continue  # spike data without position is not needed
        topdir, animal, behavior, duration = sess2topdir[sess_folder], sess2animal[sess_folder], sess2behavior[
            sess_folder], sess2duration[sess_folder]

        # Get sampling rate
        xml_path = os.path.join(sess_dir, '%s.xml' % (sess_folder))
        spike_sr, lfp_sr = get_sr_from_xml(xml_path)

        # Process theta wave
        dt = 1 / lfp_sr
        thetaband = np.array([5, 12]) * dt * 2
        deltaband = np.array([1, 4]) * dt * 2
        B, A = butter(4, thetaband, btype='band')
        Bd, Ad = butter(2, deltaband, btype='band')
        num_chs = lfp_allch.shape[0]
        thetadeltaratio = np.zeros(num_chs)
        for nch in range(num_chs):
            lfp_eachch = lfp_allch[nch, :]
            theta_filt = filtfilt(B, A, lfp_eachch)
            delta_filt = filtfilt(Bd, Ad, lfp_eachch)
            thetadeltaratio[nch] = np.sum(theta_filt ** 2) / np.sum(delta_filt ** 2)
        maxchid = np.argmax(thetadeltaratio)
        lfp_selected = lfp_allch[maxchid, :]
        tax = np.arange(lfp_selected.shape[0]) / lfp_sr
        theta_selected = filtfilt(B, A, lfp_selected)
        phase = np.angle(hilbert(theta_selected))

        wave_dict = dict()
        wave_dict['theta'] = theta_selected
        wave_dict['lfp'] = lfp_selected
        wave_dict['tax'] = tax
        wave_dict['phase'] = phase
        wavedf = pd.DataFrame(wave_dict)

        # Spike informations
        tsp_dict = {'CA1': {'ele': [], 'clu': [], 'tsp': []},
                    'CA3': {'ele': [], 'clu': [], 'tsp': []}}

        for region in ['CA1', 'CA3']:
            filtered_cell_df = cell_df[
                (cell_df.topdir == topdir) & (cell_df.animal == animal) & (cell_df.region == region) & (
                            cell_df.type == 'p')]
            region_ele_list = filtered_cell_df.ele.unique()

            for ele in region_ele_list:
                all_clu_list = filtered_cell_df[(filtered_cell_df.ele == ele) & (filtered_cell_df.type == 'p') & (
                            filtered_cell_df.fireRate < 7)].clu.unique()
                res_path = os.path.join(sess_dir, '%s.res.%d' % (sess_folder, ele))
                clu_path = os.path.join(sess_dir, '%s.clu.%d' % (sess_folder, ele))
                res = pd.read_csv(res_path, names=['tidx'])
                clu = pd.read_csv(clu_path, header=0, names=['clu_class'])
                res['time'] = res.tidx / spike_sr
                for clu_num_each in all_clu_list:
                    tsp = res.loc[clu.clu_class == clu_num_each, 'time'].to_numpy()
                    if tsp.shape[0] > 0:
                        tsp_dict[region]['ele'].append(ele)
                        tsp_dict[region]['clu'].append(clu_num_each)
                        tsp_dict[region]['tsp'].append(tsp)

        CA1_spdf = pd.DataFrame(tsp_dict['CA1'])
        CA3_spdf = pd.DataFrame(tsp_dict['CA3'])

        df_dict['animal'].append(animal)
        df_dict['topdir'].append(topdir)
        df_dict['session'].append(sess_folder)
        df_dict['duration'].append(duration)
        df_dict['behavior'].append(behavior)
        df_dict['position'].append(position_df)
        df_dict['CA1_spdf'].append(CA1_spdf)
        df_dict['CA3_spdf'].append(CA3_spdf)
        df_dict['wave'].append(wavedf)
        df_dict['spike_samplingrate'].append(spike_sr)
        df_dict['lfp_samplingrate'].append(lfp_sr)
        df_dict['position_samplingrate'].append(position_fs)

hc3data_df = pd.DataFrame(df_dict)
hc3data_df.to_pickle('data/crcns-hc-3/hc3data_df_raw.pickle')

38)i01_maze15/i01_maze15_MS.001 .whl or . mat not found
48)Skip data/crcns-hc-3/hc3data_df_raw3.pickle
59)Skip data/crcns-hc-3/hc3data_df_raw4.pickle
62)Skip data/crcns-hc-3/hc3data_df_raw2.pickle
64)Skip data/crcns-hc-3/hc3data_df_raw1.pickle
76)

# Cleaning
1. Check sampling rates by duration, positions by pixel
2. Interpolate missing positions
3. Convert x, y coordinates from pixel to cm 
4. Concatenate sessions within topdir (Optional)


In [3]:
# 1. Check sampling rates by duration, positions by pixel
check_dict = dict(beh=[], xmax=[], ymax=[], xrange=[], yrange=[], duration=[], tmaxRun=[], tspmaxCA1=[],
                  tspmaxCA3=[], tmaxWave=[])
count_dict = dict(topdir=[], session=[], CA1_ncells=[], CA3_ncells=[])
clu_all = []
for i in range(hc3data_df.shape[0]):

    posdf = hc3data_df.loc[i, 'position']
    enclosure, duration = hc3data_df.loc[i, ['behavior', 'duration']]
    topdir, session = hc3data_df.loc[i, ['topdir', 'session']]

    wave = hc3data_df.loc[i, 'wave']

    wave_tmax, wave_tmin = wave['tax'].max(), wave['tax'].min()

    xmax, ymax, tmax = posdf['x'].max(), posdf['y'].max(), posdf['t'].max()

    nonnanmask = (posdf['x'] > -1) & (posdf['y'] > -1)

    xmin, ymin = posdf.loc[nonnanmask, 'x'].min(), posdf.loc[nonnanmask, 'y'].min()

    CA1spdf, CA3spdf = hc3data_df.loc[i, ['CA1_spdf', 'CA3_spdf']]

    if CA1spdf.shape[0] > 0:
        tmaxCA1 = np.concatenate(CA1spdf['tsp'].to_list()).max()

        clu_all.extend(CA1spdf['clu'].to_list())

    else:
        tmaxCA1 = np.nan
    if CA3spdf.shape[0] > 0:
        tmaxCA3 = np.concatenate(CA3spdf['tsp'].to_list()).max()
        clu_all.extend(CA1spdf['clu'].to_list())
    else:
        tmaxCA3 = np.nan

    check_dict['beh'].append(enclosure)
    check_dict['xmax'].append((xmin, xmax))
    check_dict['ymax'].append((ymin, ymax))
    check_dict['xrange'].append(xmax - xmin)
    check_dict['yrange'].append(ymax - ymin)
    check_dict['duration'].append(duration)
    check_dict['tmaxRun'].append(tmax)
    check_dict['tspmaxCA1'].append(tmaxCA1)
    check_dict['tspmaxCA3'].append(tmaxCA3)
    check_dict['tmaxWave'].append(wave_tmax)
    count_dict['topdir'].append(topdir)
    count_dict['session'].append(session)
    count_dict['CA1_ncells'].append(CA1spdf.shape[0])
    count_dict['CA3_ncells'].append(CA3spdf.shape[0])

check_df = pd.DataFrame(check_dict)
count_df = pd.DataFrame(count_dict)

unique_topdirs = count_df.topdir.unique()
selected_idx = []
for topdir in unique_topdirs:
    tmpdf = count_df[count_df['topdir'] == topdir]
    maxidx = tmpdf[tmpdf['CA3_ncells'] == tmpdf['CA3_ncells'].max()].index
    if maxidx.shape[0] > 1:
        maxidx = [maxidx[0]]

    selected_idx.extend(maxidx)

num_allCA1 = count_df.loc[selected_idx, 'CA1_ncells'].sum()
num_allCA3 = count_df.loc[selected_idx, 'CA3_ncells'].sum()

print('With unique topdir, num CA1 = %d, num CA3 = %d' % (num_allCA1, num_allCA3))
print('Unique clu: ', np.unique(clu_all))
print(check_df)

With unique topdir, num CA1 = 1115, num CA3 = 345
Unique clu:  [ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25]


In [ ]:
hc3data_df = pd.read_pickle('data/crcns-hc-3/hc3data_df_raw.pickle')

In [4]:
# 1. Interpolate missing positions
# 2. rescale x, y coordinates from pixel to cm
print('Cleaning position data')
arena_l = {'bigSquare': 180, 'midSquare': 120}
new_posdf_list = []
for i in range(hc3data_df.shape[0]):
    posdf = hc3data_df.loc[i, 'position']
    beh = hc3data_df.loc[i, 'behavior']
    new_posdf = process_position_df(posdf, arena_l=arena_l[beh])
    new_posdf_list.append(new_posdf)
hc3data_df['posdf'] = new_posdf_list


In [146]:
# 4. Concatenate sessions within topdir (Optional)
combined_df_dict = dict(animal=[],
                        topdir=[],
                        session=[],
                        duration=[],
                        behavior=[],
                        CA1_spdf=[],
                        CA3_spdf=[],
                        wave=[],
                        posdf=[],
                        spike_samplingrate=[],
                        lfp_samplingrate=[],
                        position_samplingrate=[])


for topdir, topdf in hc3data_df.groupby('topdir'):
    if topdf.shape[0] < 2:  # Nothing to combine
        for col in combined_df_dict.keys():
            combined_df_dict[col].append(topdf.iloc[0][col])
    else:
        topdf = topdf.sort_values('session').reset_index(drop=True)
        num_sess = topdf.shape[0]
        animal, beh, ssr, lsr, psr = topdf.loc[0, ['animal', 'behavior' , 'spike_samplingrate', 'lfp_samplingrate', 'position_samplingrate']]
        
        
        # Shift time
        posdf_list, wave_list, CA1_spdf_list, CA3_spdf_list = [], [], [], []
        shifttime = 0
        for nsess in range(num_sess):
            posdf, CA1_spdf, CA3_spdf, wave = topdf.loc[nsess, ['posdf', 'CA1_spdf', 'CA3_spdf', 'wave']]
            pos_tmin, pos_tmax = posdf['t'].min(), posdf['t'].max()
            wave = wave[(wave['tax']>pos_tmin) & (wave['tax']< pos_tmax)].reset_index(drop=True)
            
            posdf['t'] = posdf['t'] - pos_tmin + shifttime
            wave['tax'] = wave['tax'] - pos_tmin + shifttime
            
            
            CA1nums, CA3nums = CA1_spdf.shape[0], CA3_spdf.shape[0]
            tspca1_list, tspca3_list = [], []
            for ica1 in range(CA1nums):
                tspca1 = CA1_spdf.loc[ica1, 'tsp']
                tspca1 = tspca1[(tspca1 > pos_tmin) & (tspca1 < pos_tmax)]
                tspca1 = tspca1 - pos_tmin + shifttime
                tspca1_list.append(tspca1)
                
            for ica3 in range(CA3nums):
                tspca3 = CA3_spdf.loc[ica3, 'tsp']
                tspca3 = tspca3[(tspca3 > pos_tmin) & (tspca3 < pos_tmax)]
                tspca3 = tspca3 - pos_tmin + shifttime
                tspca3_list.append(tspca3)
            CA1_spdf['tsp'] = tspca1_list
            CA3_spdf['tsp'] = tspca3_list
            
            posdf_list.append(posdf)
            wave_list.append(wave)            
            CA1_spdf_list.append(CA1_spdf)
            CA3_spdf_list.append(CA3_spdf)
            
            shifttime = posdf['t'].max() + (1/psr)
            
        # Concatenate
        poscat = pd.concat(posdf_list, ignore_index=True, axis=0)
        wavecat = pd.concat(wave_list, ignore_index=True, axis=0)
        duration = poscat['t'].max() - poscat['t'].min()
        ca1dict = {'ele':[], 'clu':[], 'tsp':[]}
        CA1_spdf_cat = pd.concat(CA1_spdf_list, ignore_index=True, axis=0)
        CA1_spdf_cat['cell_code'] = CA1_spdf_cat['ele']*1000 + CA1_spdf_cat['clu']
        for cell_code, codedf in CA1_spdf_cat.groupby('cell_code'):
            ele = codedf.iloc[0]['ele']
            clu = codedf.iloc[0]['clu']
            tsp_cat = np.concatenate(codedf['tsp'].to_list())
            ca1dict['ele'].append(ele)
            ca1dict['clu'].append(clu)
            ca1dict['tsp'].append(tsp_cat)
            
        new_CA1_spdf = pd.DataFrame(ca1dict)
        
        ca3dict = {'ele':[], 'clu':[], 'tsp':[]}
        CA3_spdf_cat = pd.concat(CA3_spdf_list, ignore_index=True, axis=0)
        CA3_spdf_cat['cell_code'] = CA3_spdf_cat['ele']*1000 + CA3_spdf_cat['clu']
        for cell_code, codedf in CA3_spdf_cat.groupby('cell_code'):
            ele = codedf.iloc[0]['ele']
            clu = codedf.iloc[0]['clu']
            tsp_cat = np.concatenate(codedf['tsp'].to_list())
            ca3dict['ele'].append(ele)
            ca3dict['clu'].append(clu)
            ca3dict['tsp'].append(tsp_cat)
            
        new_CA3_spdf = pd.DataFrame(ca1dict)

        
        combined_df_dict['animal'].append(animal)
        combined_df_dict['topdir'].append(topdir)
        combined_df_dict['session'].append('+'.join(topdf.session.to_list()))
        combined_df_dict['duration'].append(duration)
        combined_df_dict['behavior'].append(beh)
        combined_df_dict['CA1_spdf'].append(new_CA1_spdf)
        combined_df_dict['CA3_spdf'].append(new_CA3_spdf)
        combined_df_dict['wave'].append(wavecat)
        combined_df_dict['posdf'].append(poscat)
        combined_df_dict['spike_samplingrate'].append(ssr)
        combined_df_dict['lfp_samplingrate'].append(lsr)
        combined_df_dict['position_samplingrate'].append(psr)
        
combined_df = pd.DataFrame(combined_df_dict) 
        

In [148]:
combined_df.to_pickle('data/crcns-hc-3/hc3data_df_raw1.pickle')

# Compute occupancy


In [2]:
hc3data_df = pd.read_pickle('data/crcns-hc-3/hc3data_df_raw1.pickle')

In [ ]:
# Compute occupancy
print('Compute occupancy')
# # Compute occupancy
# It takes ~4 hours. Recommend saving the result as checkpoint.
num_sess = hc3data_df.shape[0]
occ_list = []
for nsess in range(num_sess):
    print("%d/%d Session - Computing occupancy" % (nsess, num_sess))

    posdf = hc3data_df.loc[nsess, 'posdf']
    x, y, t = posdf.x.to_numpy(), posdf.y.to_numpy(), posdf.t.to_numpy()
    xx, yy, occupancy = get_occupancy(x, y, t)
    occ_list.append(dict(xx=xx, yy=yy, occ=occupancy))
hc3data_df['occ_dict'] = occ_list

In [ ]:
hc3data_df = pd.to_pickle('data/crcns-hc-3/hc3data_df_raw2.pickle')

# Shuffle Spatial information & Create Rate map

In [7]:
hc3data_df = pd.read_pickle('data/crcns-hc-3/hc3data_df_raw2.pickle')


In [ ]:
print('Compute rate map and shuffle spatial information')

# Shuffling
Nshuffle = 1000
seed = 0
num_sess = hc3data_df.shape[0]
for nsess in range(num_sess):

    posdf = hc3data_df.loc[nsess, 'posdf']
    x, y, t = posdf.x.to_numpy(), posdf.y.to_numpy(), posdf.t.to_numpy()
    trange = (t.max(), t.min())
    x_interp = interp1d(t, x)
    y_interp = interp1d(t, y)
    x_ax, y_ax, occupancy = get_occupancy_bins(x, y, t, binstep=10)

    occ_kde_dict = hc3data_df.loc[nsess, 'occ_dict']
    xx_kde, yy_kde, occupancy_kde = occ_kde_dict['xx'], occ_kde_dict['yy'], occ_kde_dict['occ']

    for ca in ['CA1', 'CA3']:
        info_list = []
        info_shuf_list = []
        info_pval_list = []
        pf_list = []

        spdf = hc3data_df.loc[nsess, '%s_spdf' % (ca)]
        if spdf.shape[0] < 1:
            spdf['spatialinfo_perspike'] = info_list
            spdf['spatialinfo_perspike_shuf'] = info_shuf_list
            spdf['spatialinfo_pval'] = info_pval_list
            spdf['pf'] = pf_list
            continue

        num_cells = spdf.shape[0]
        for ncell in range(num_cells):
            print('%d/%d Session, %s %d/%d cell ' % (nsess, num_sess, ca, ncell, num_cells), end='')
            tsp = spdf.loc[ncell, 'tsp']
            tsp_in = tsp[(tsp < trange[0]) & (tsp > trange[1])]
            if tsp_in.shape[0] < 2:
                print('Skip, tsp.shape[0]=%d' % (tsp_in.shape[0]))
                info_list.append(np.nan)
                info_shuf_list.append(np.nan)
                info_pval_list.append(np.nan)
                pf_list.append(np.nan)
                continue

            xsp_in = x_interp(tsp_in)
            ysp_in = y_interp(tsp_in)
            freq, rate = placefield_bins(x_ax, y_ax, occupancy, xsp_in, ysp_in)
            info_persecond, info_perspike = spatial_information(rate, occupancy)

            freq_kde, rate_kde = placefield(xx_kde, yy_kde, occupancy_kde, xsp_in, ysp_in, tsp_in)

            # Shuffling
            info_perspike_shuffleds = np.zeros(Nshuffle)
            for nshuf in range(Nshuffle):
                tsp_in_min = tsp_in.min()
                tsp_in_offseted = tsp_in - tsp_in_min
                maxtsp_offseted = tsp_in_offseted.max()
                np.random.seed(seed)
                shiftamount = np.random.uniform(0, maxtsp_offseted)
                tsp_in_shuffled_offseted = np.mod(tsp_in_offseted + shiftamount, maxtsp_offseted)
                tsp_in_shuffled = tsp_in_shuffled_offseted + tsp_in_min

                tsp_in_shuffled = tsp_in_shuffled[(tsp_in_shuffled < trange[0]) & (tsp_in_shuffled > trange[1])]
                xsp_in_shuffled = x_interp(tsp_in_shuffled)
                ysp_in_shuffled = y_interp(tsp_in_shuffled)
                freq_shuffled, rate_shuffled = placefield_bins(x_ax, y_ax, occupancy, xsp_in_shuffled,
                                                               ysp_in_shuffled)
                _, info_perspike_shuffleds[nshuf] = spatial_information(rate_shuffled, occupancy)

                seed += 1

            try:
                pval = 1 - (info_perspike > info_perspike_shuffleds).mean()
            except:
                pval = 1

            info_list.append(info_perspike)
            info_shuf_list.append(info_perspike_shuffleds)
            info_pval_list.append(pval)
            pf_list.append(dict(freq=freq_kde, rate=rate_kde))
            print('pval=%0.4f' % (pval))
        spdf['spatialinfo_perspike'] = info_list
        spdf['spatialinfo_perspike_shuf'] = info_shuf_list
        spdf['spatialinfo_pval'] = info_pval_list
        spdf['pf'] = pf_list



In [ ]:
hc3data_df.to_pickle('data/crcns-hc-3/hc3data_df_raw3.pickle')

# Segment and filter fields

In [8]:
hc3data_df = pd.read_pickle('data/crcns-hc-3/hc3data_df_raw3.pickle')


In [9]:
# # Segment place fields & Filter place fields

print('Segment and filter single fields')
# # Segment place fields & Filter place fields
bounddict = dict(bigSquare=(0, 180), midSquare=(0, 120))
areathresh_dict = dict(bigSquare=200, midSquare=120)
num_sess = hc3data_df.shape[0]

CA_fielddf_list = dict(CA1=[], CA3=[])
for nsess in range(num_sess):
    print('%d/%d Session' % (nsess, num_sess))

    behavior = hc3data_df.loc[nsess, 'behavior']
    xybound = bounddict[behavior]
    occ_dict = hc3data_df.loc[nsess, 'occ_dict']
    xx, yy, occ = occ_dict['xx'], occ_dict['yy'], occ_dict['occ']

    for ca in ['CA1', 'CA3']:

        fielddf_dict = dict(cellid=[], mask=[], xyval=[])
        spdf = hc3data_df.loc[nsess, '%s_spdf' % (ca)]
        drop_mask = (spdf['spatialinfo_pval'] > 0.05) | (spdf['pf'].isna())
        spdf.drop(index=spdf[drop_mask].index, axis=0, inplace=True)
        spdf.index = np.arange(spdf.shape[0])

        num_cells = spdf.shape[0]

        for ncell in range(num_cells):
            rate = spdf.loc[ncell, 'pf']['rate']
            normmap = rate / rate.max()
            contours = find_contours(normmap, level=0.2)
            field_dict = dict(mask=[], xyval=[])

            for c_each in contours:
                c_rowi = c_each[:, 0]  # y
                c_coli = c_each[:, 1]  # x

                c_coli, c_rowi, case = complete_contourline(c_coli, c_rowi, xybound, xybound)

                mask = polygon2mask(xx.shape, np.stack([c_rowi, c_coli]).T)  # Remember to transpose!

                c_rowi = np.around(c_rowi).astype(int)
                c_coli = np.around(c_coli).astype(int)
                c_x = xx[c_rowi, c_coli]
                c_y = yy[c_rowi, c_coli]
                xyval = np.stack([c_x, c_y]).T

                area = mask.sum()
                if area < 1:
                    continue
                masked_rate = rate[mask]
                meanrate_in = np.mean(masked_rate)
                meanrate_out = np.mean(rate[np.invert(mask)])
                peak_rate_in = masked_rate.max()

                if (area > areathresh_dict[behavior]) and (meanrate_in > meanrate_out) and (peak_rate_in > 1):
                    fielddf_dict['cellid'].append(ncell)
                    fielddf_dict['mask'].append(mask)
                    fielddf_dict['xyval'].append(xyval)

        CA_fielddf_list[ca].append(pd.DataFrame(fielddf_dict))

hc3data_df['CA1fields'] = CA_fielddf_list['CA1']
hc3data_df['CA3fields'] = CA_fielddf_list['CA3']



0/47 Session
1/47 Session
2/47 Session
3/47 Session
4/47 Session
5/47 Session
6/47 Session
7/47 Session
8/47 Session
9/47 Session
10/47 Session
11/47 Session
12/47 Session
13/47 Session
14/47 Session
15/47 Session
16/47 Session
17/47 Session
18/47 Session
19/47 Session
20/47 Session
21/47 Session
22/47 Session
23/47 Session
24/47 Session
25/47 Session
26/47 Session
27/47 Session
28/47 Session
29/47 Session
30/47 Session
31/47 Session
32/47 Session
33/47 Session
34/47 Session
35/47 Session
36/47 Session
37/47 Session
38/47 Session
39/47 Session
40/47 Session
41/47 Session
42/47 Session
43/47 Session
44/47 Session
45/47 Session
46/47 Session


In [ ]:
print('Choose one sessions within each topdir')
# # (Optional) if you didn't concatenate the sub-sessions, then choose one session from each topdir, or do nothing
# # Choose one session from each topdir, based on max CA3 field count, otherwise max CA1 field count
selected_dfindex = []
for topdir_each in hc3data_df.topdir.unique():
    topdirdf = hc3data_df[hc3data_df['topdir'] == topdir_each]
    num_sess = topdirdf.shape[0]
    idx_arr = np.zeros(num_sess)
    count1_arr = np.zeros(num_sess)
    count3_arr = np.zeros(num_sess)
    for arri, dfidx in enumerate(topdirdf.index):
        ca1spdf = topdirdf.loc[dfidx, 'CA1fields']
        ca3spdf = topdirdf.loc[dfidx, 'CA3fields']

        count_ca1 = ca1spdf.shape[0]
        count_ca3 = ca3spdf.shape[0]

        idx_arr[arri] = dfidx
        count1_arr[arri] = count_ca1
        count3_arr[arri] = count_ca3

    maxcount3 = np.max(count3_arr)
    if maxcount3 < 1:
        maxarrid = np.argmax(count1_arr)
    else:
        maxarrid = np.argmax(count3_arr)

    maxdfid = idx_arr[maxarrid]
    selected_dfindex.append(int(maxdfid))

hc3data_df = hc3data_df.loc[selected_dfindex].reset_index(drop=True)

# Show field counts
CA1spdf = pd.concat(hc3data_df['CA1fields'].to_list(), axis=0, ignore_index=True)
CA3spdf = pd.concat(hc3data_df['CA3fields'].to_list(), axis=0, ignore_index=True)
print('Field count CA1 = %d, CA3 = %d' % (CA1spdf.shape[0], CA3spdf.shape[0]))

In [ ]:
hc3data_df.to_pickle('data/crcns-hc-3/hc3data_df_raw4.pickle')

# Identify pairs

In [3]:
hc3data_df = pd.read_pickle('data/crcns-hc-3/hc3data_df_raw4.pickle')

In [4]:
print('Identify pairs')
num_sess = hc3data_df.shape[0]

CA1pairs_list = []
CA3pairs_list = []

for nsess in range(num_sess):

    posdf, occ_dict = hc3data_df.loc[nsess, ['posdf', 'occ_dict']]
    x, y, t = posdf['x'].to_numpy(), posdf['y'].to_numpy(), posdf['t'].to_numpy()
    trange = (t.max(), t.min())
    xinterp, yinterp = interp1d(t, x), interp1d(t, y)
    xx, yy = occ_dict['xx'], occ_dict['yy']
    x_ax, y_ax = xx[0, :], yy[:, 0]
    pair_dict = dict(CA1=dict(field1_id=[], field2_id=[]),
                     CA3=dict(field1_id=[], field2_id=[]))
    for ca in ['CA1', 'CA3']:
        fielddf = hc3data_df.loc[nsess, '%sfields' % ca]
        spdf = hc3data_df.loc[nsess, '%s_spdf' % ca]

        num_fields = fielddf.shape[0]
        if num_fields < 0:
            continue

        for fieldid1 in range(num_fields):
            for fieldid2 in range(fieldid1 + 1, num_fields):
                print('\r%d/%d Session Pair %d - %d' % (nsess, num_sess, fieldid1, fieldid2), flush=True, end='')
                cellid1, mask1 = fielddf.loc[fieldid1, ['cellid', 'mask']]
                tsp1, pf1 = spdf.loc[cellid1, ['tsp', 'pf']]
                tsp_in1 = tsp1[(tsp1 < trange[0]) & (tsp1 > trange[1])]
                xsp_in1, ysp_in1 = xinterp(tsp_in1), yinterp(tsp_in1)
                ind_x1 = np.argmin(np.square(pair_diff(xsp_in1, x_ax)), axis=1)
                ind_y1 = np.argmin(np.square(pair_diff(ysp_in1, y_ax)), axis=1)
                tok1 = np.zeros(ind_x1.shape[0]).astype(bool)
                for i in range(ind_x1.shape[0]):
                    tok1[i] = mask1[ind_y1[i], ind_x1[i]]
                tsp_inmask1 = tsp_in1[tok1]

                cellid2, mask2 = fielddf.loc[fieldid2, ['cellid', 'mask']]
                tsp2, pf2 = spdf.loc[cellid2, ['tsp', 'pf']]
                tsp_in2 = tsp2[(tsp2 < trange[0]) & (tsp2 > trange[1])]
                xsp_in2, ysp_in2 = xinterp(tsp_in2), yinterp(tsp_in2)
                ind_x2 = np.argmin(np.square(pair_diff(xsp_in2, x_ax)), axis=1)
                ind_y2 = np.argmin(np.square(pair_diff(ysp_in2, y_ax)), axis=1)
                tok2 = np.zeros(ind_x2.shape[0]).astype(bool)
                for i in range(ind_x2.shape[0]):
                    tok2[i] = mask2[ind_y2[i], ind_x2[i]]
                tsp_inmask2 = tsp_in2[tok2]

                pairtsp_idx1, pairtsp_idx2 = find_pair_times(tsp_inmask1, tsp_inmask2)

                num_pairedspikes = pairtsp_idx1.shape[0] + pairtsp_idx2.shape[0]
                mask_and = mask1 & mask2
                if (num_pairedspikes > 16) & (mask_and.sum() > 0):
                    pair_dict[ca]['field1_id'].append(fieldid1)
                    pair_dict[ca]['field2_id'].append(fieldid2)

    CA1pairs_list.append(pd.DataFrame(pair_dict['CA1']))
    CA3pairs_list.append(pd.DataFrame(pair_dict['CA3']))

hc3data_df['CA1pairs'] = CA1pairs_list
hc3data_df['CA3pairs'] = CA3pairs_list
print()
print('Number of pairs in CA1 = %s' % pd.concat(hc3data_df['CA1pairs'].to_list(), axis=0).shape[0])
print('Number of pairs in CA3 = %s' % pd.concat(hc3data_df['CA3pairs'].to_list(), axis=0).shape[0])

46/47 Session Pair 26 - 2713(4910, 2)
(2366, 2)


In [30]:
hc3data_df.to_pickle('data/crcns-hc-3/hc3data_df_processed.pickle')

# Check fields and pairs

In [2]:
hc3rawdf = pd.read_pickle('data/crcns-hc-3/hc3data_df_raw.pickle')

In [4]:
hc3rawdf.loc[0, 'CA1_spdf']

,ele,clu,tsp
0,1,2,"[2.0528, 3.4549, 6.4071, 6.43055, 68.62355, 72..."
1,1,3,"[7.29045, 36.1801, 51.1306, 79.27325, 87.58035..."
2,1,4,"[176.143, 181.24785, 192.94745, 192.95255, 192..."
3,1,5,"[0.16345, 0.25915, 0.2653, 0.6112, 0.6169, 0.7..."
4,1,6,"[8.4167, 14.83185, 14.8395, 14.8454, 14.85525,..."
5,1,7,"[28.97045, 29.1192, 35.5983, 47.31515, 75.5565..."
6,1,8,"[24.0364, 24.04135, 24.04545, 24.0491, 24.055,..."
7,1,9,"[147.42405, 147.6226, 151.02665, 165.6056, 165..."
8,1,11,"[4.9391, 13.9483, 24.9444, 25.95865, 26.1235, ..."
9,1,12,"[35.62525, 81.41815, 81.42615, 81.57765, 81.65..."


In [ ]:
hc3data_df = pd.read_pickle('data/crcns-hc-3/hc3processed_chooseOneSession.pickle')

In [ ]:
# Check single

# Check pairs
nsess = 0
ca = 'CA1'

num_sess = hc3data_df.shape[0]


for nsess in range(num_sess):
    posdf, occ_dict = hc3data_df.loc[nsess, ['posdf', 'occ_dict']]
    x, y, t = posdf['x'].to_numpy(), posdf['y'].to_numpy(), posdf['t'].to_numpy()
    trange = (t.max(), t.min())
    xinterp, yinterp = interp1d(t, x), interp1d(t, y)
    xx, yy = occ_dict['xx'], occ_dict['yy']
    x_ax, y_ax = xx[0, :], yy[:, 0]
    pair_dict = dict(CA1=dict(field1_id=[], field2_id=[]),
                     CA3=dict(field1_id=[], field2_id=[]))
    fielddf = hc3data_df.loc[nsess, '%sfields'%ca]
    spdf = hc3data_df.loc[nsess, '%s_spdf'%ca]
    
    for nfield in range(fielddf.shape[0]):
        cellid, mask, xyval = fielddf.loc[nfield, ['cellid', 'mask', 'xyval']]
        
        pf, spinfo, pval = spdf.loc[cellid, ['pf', 'spatialinfo_perspike', 'spatialinfo_pval']]
        
        
        
        
        fig, ax = plt.subplots(1, 3, figsize=(15, 5))
        ax[0].pcolormesh(xx, yy, pf['rate'], shading='auto')
        ax[1].pcolormesh(xx, yy, mask, shading='auto')
        ax[0].plot(xyval[:, 0], xyval[:, 1], c='r')
        ax[1].plot(xyval[:, 0], xyval[:, 1], c='r')
        
        
        
        fig.suptitle('%s session, field %d (cell %d) spinfo=%0.2f(p=%0.4f)'%(nsess, nfield, cellid, spinfo, pval))
        
        

In [ ]:
# Check pairs
nsess = 26
ca = 'CA3'

num_sess = hc3data_df.shape[0]


for nsess in range(num_sess):
    posdf, occ_dict = hc3data_df.loc[nsess, ['posdf', 'occ_dict']]
    x, y, t = posdf['x'].to_numpy(), posdf['y'].to_numpy(), posdf['t'].to_numpy()
    trange = (t.max(), t.min())
    xinterp, yinterp = interp1d(t, x), interp1d(t, y)
    xx, yy = occ_dict['xx'], occ_dict['yy']
    x_ax, y_ax = xx[0, :], yy[:, 0]
    pair_dict = dict(CA1=dict(field1_id=[], field2_id=[]),
                     CA3=dict(field1_id=[], field2_id=[]))
    fielddf = hc3data_df.loc[nsess, '%sfields'%ca]
    spdf = hc3data_df.loc[nsess, '%s_spdf'%ca]
    pairdf = hc3data_df.loc[nsess, '%spairs'%ca]

    num_pairs = pairdf.shape[0]
    for npair in range(num_pairs):
        if npair > 20:
            raise
        f1id, f2id = pairdf.loc[npair, ['field1_id', 'field2_id']]

        cellid1, mask1, xyval1 = fielddf.loc[f1id, ['cellid', 'mask', 'xyval']]
        cellid2, mask2, xyval2 = fielddf.loc[f2id, ['cellid', 'mask', 'xyval']]

        fig, ax = plt.subplots(1, 3, figsize=(15, 5))
        ax[0].pcolormesh(xx, yy, mask1, shading='auto')
        ax[1].pcolormesh(xx, yy, mask2, shading='auto')
        ax[2].pcolormesh(xx, yy, mask1 & mask2, shading='auto')
        ax[2].plot(xyval1[:, 0], xyval1[:, 1], c='r')
        ax[2].plot(xyval2[:, 0], xyval2[:, 1], c='b')
        fig.suptitle('%s session, pair %d (field %d & %d) (cell %d & %d)'%(nsess, npair, f1id, f2id, cellid1, cellid2))


In [10]:
df = hc3data_df

In [56]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import join
from scipy.interpolate import interp1d
from pycircstat.descriptive import resultant_vector_length, cdiff
from pycircstat.descriptive import mean as circmean
import warnings

from common.utils import load_pickle
from common.comput_utils import check_border, IndataProcessor, midedges, segment_passes, \
    check_border_sim, append_info_from_passes, \
    DirectionerBining, DirectionerMLM, timeshift_shuffle_exp_wrapper, circular_density_1d, get_numpass_at_angle, \
    PassShuffler

from common.visualization import color_wheel, directionality_polar_plot, customlegend

from common.script_wrappers import DirectionalityStatsByThresh, PrecessionProcesser, PrecessionFilter, \
    construct_passdf_sim, get_single_precessdf, compute_precessangle
from common.shared_vars import fontsize, ticksize, legendsize, titlesize, ca_c, dpi, total_figw

vthresh = 5
sthresh = 3
NShuffles = 200

fielddf_dict = dict(ca=[], num_spikes=[], border=[], aver_rate=[], peak_rate=[],
                    rate_angle=[], rate_R=[], rate_R_pval=[], field_area=[], field_bound=[],
                    precess_df=[], precess_angle=[], precess_angle_low=[], precess_R=[], precess_R_pval=[],
                    numpass_at_precess=[], numpass_at_precess_low=[])

num_sess = df.shape[0]

aedges = np.linspace(-np.pi, np.pi, 36)
abind = aedges[1] - aedges[0]
sp_binwidth = 5

aedges_precess = np.linspace(-np.pi, np.pi, 6)
kappa_precess = 1
precess_filter = PrecessionFilter()
nspikes_stats = {'CA1': 6, 'CA3': 7}  # 25% quantile for 0.4t
for nsess in range(num_sess):

    posdf = df.loc[nsess, 'posdf']
    occ_dict = df.loc[nsess, 'occ_dict']
    xx, yy = occ_dict['xx'], occ_dict['yy']
    x_ax, y_ax = xx[0, :], yy[:, 0]
    tunner = IndataProcessor(posdf, vthresh=vthresh, sthresh=sthresh, minpasstime=0.4)
    trange = (tunner.t.max(), tunner.t.min())
    dt = tunner.t[1] - tunner.t[0]
    interpolater_angle = interp1d(tunner.t, tunner.angle)
    interpolater_x = interp1d(tunner.t, tunner.x)
    interpolater_y = interp1d(tunner.t, tunner.y)

    wave = df.loc[nsess, 'wave']
    wave = {col:np.array(wave[col]) for col in wave.columns}
    precesser = PrecessionProcesser(wave=wave)
    precesser.set_trange(trange)

    for ca in ['CA1', 'CA3']:

        # Get data
        field_df, spdf = df.loc[nsess, ['%sfields' % (ca), '%s_spdf' % (ca)]]
        num_fields = field_df.shape[0]

        for nfield in range(num_fields):
            print('%d/%d trial, %s, %d/%d field' % (nsess, num_sess, ca, nfield, num_fields))
            cellid, mask, xyval = field_df.loc[nfield, ['cellid', 'mask', 'xyval']]
            tsp, pf = spdf.loc[cellid, ['tsp', 'pf']]

            # Get field info
            field_area = np.sum(mask)
            field_d = np.sqrt(field_area / np.pi) * 2
            border = check_border(mask, margin=2)

            # Construct passes (segment & chunk)
            tok, idin = tunner.get_idin(mask.T, x_ax, y_ax)
            passdf = tunner.construct_singlefield_passdf(tok, tsp, interpolater_x, interpolater_y,
                                                         interpolater_angle)

            allchunk_df = passdf[(~passdf['rejected']) & (passdf['chunked'] < 2)].reset_index(drop=True)

            # Get info from passdf and interpolate
            if allchunk_df.shape[0] < 1:
                continue
            all_x_list, all_y_list = allchunk_df['x'].to_list(), allchunk_df['y'].to_list()
            all_t_list, all_passangles_list = allchunk_df['t'].to_list(), allchunk_df['angle'].to_list()
            all_tsp_list, all_chunked_list = allchunk_df['tsp'].to_list(), allchunk_df['chunked'].to_list()
            all_x = np.concatenate(all_x_list)
            all_y = np.concatenate(all_y_list)
            all_passangles = np.concatenate(all_passangles_list)
            all_tsp = np.concatenate(all_tsp_list)
            all_anglesp = np.concatenate(allchunk_df['spikeangle'].to_list())
            xsp, ysp = np.concatenate(allchunk_df['spikex'].to_list()), np.concatenate(
                allchunk_df['spikey'].to_list())
            pos = np.stack([all_x, all_y]).T
            possp = np.stack([xsp, ysp]).T

            # Average firing rate
            aver_rate = all_tsp.shape[0] / (all_x.shape[0] * dt)
            peak_rate = np.max(pf['rate'] * mask)

            # Field's directionality - need angle, anglesp, pos
            num_spikes = all_tsp.shape[0]
            occ_bins, _ = np.histogram(all_passangles, bins=aedges)
            mlmer = DirectionerMLM(pos, all_passangles, dt, sp_binwidth=sp_binwidth, a_binwidth=abind)
            rate_angle, rate_R, norm_prob_mlm = mlmer.get_directionality(possp, all_anglesp)

            # Time shift shuffling for rate directionality
            rate_R_pval = timeshift_shuffle_exp_wrapper(all_tsp_list, all_t_list, rate_R,
                                                        NShuffles, mlmer, interpolater_x,
                                                        interpolater_y, interpolater_angle, trange)

            # Precession per pass
            neuro_keys_dict = dict(tsp='tsp', spikev='spikev', spikex='spikex', spikey='spikey',
                                   spikeangle='spikeangle')
            accept_mask = (~passdf['rejected']) & (passdf['chunked'] < 2)
            passdf['excluded_for_precess'] = ~accept_mask
            precessdf, precess_angle, precess_R, _ = get_single_precessdf(passdf, precesser, precess_filter,
                                                                          neuro_keys_dict,
                                                                          field_d=field_d, kappa=kappa_precess,
                                                                          bins=None)
            fitted_precessdf = precessdf[precessdf['fitted']].reset_index(drop=True)
            # Proceed only if precession exists
            if (precess_angle is not None) and (fitted_precessdf['precess_exist'].sum() > 0):

                # Post-hoc precession exclusion
                _, binR, postdoc_dens = compute_precessangle(
                    pass_angles=fitted_precessdf['mean_anglesp'].to_numpy(),
                    pass_nspikes=fitted_precessdf['pass_nspikes'].to_numpy(),
                    precess_mask=fitted_precessdf['precess_exist'].to_numpy(),
                    kappa=None, bins=aedges_precess)
                (_, passbins_p, passbins_np, _) = postdoc_dens
                all_passbins = passbins_p + passbins_np
                numpass_at_precess = get_numpass_at_angle(target_angle=precess_angle, aedge=aedges_precess,
                                                          all_passbins=all_passbins)

                # Precession - low-spike passes
                ldf = fitted_precessdf[fitted_precessdf['pass_nspikes'] < nspikes_stats[ca]]  # 25% quantile
                if (ldf.shape[0] > 0) and (ldf['precess_exist'].sum() > 0):
                    precess_angle_low, _, _ = compute_precessangle(pass_angles=ldf['mean_anglesp'].to_numpy(),
                                                                   pass_nspikes=ldf['pass_nspikes'].to_numpy(),
                                                                   precess_mask=ldf['precess_exist'].to_numpy(),
                                                                   kappa=kappa_precess, bins=None)
                    _, _, postdoc_dens_low = compute_precessangle(pass_angles=ldf['mean_anglesp'].to_numpy(),
                                                                  pass_nspikes=ldf['pass_nspikes'].to_numpy(),
                                                                  precess_mask=ldf['precess_exist'].to_numpy(),
                                                                  kappa=None, bins=aedges_precess)
                    (_, passbins_p_low, passbins_np_low, _) = postdoc_dens_low
                    all_passbins_low = passbins_p_low + passbins_np_low
                    numpass_at_precess_low = get_numpass_at_angle(target_angle=precess_angle_low,
                                                                  aedge=aedges_precess,
                                                                  all_passbins=all_passbins_low)
                else:
                    precess_angle_low = None
                    numpass_at_precess_low = None

                precess_R_pval = 1


            else:
                numpass_at_precess = None
                precess_angle_low = None
                numpass_at_precess_low = None
                precess_R_pval = None

            fielddf_dict['ca'].append(ca)
            fielddf_dict['num_spikes'].append(num_spikes)
            fielddf_dict['field_area'].append(field_area)
            fielddf_dict['field_bound'].append(xyval)
            fielddf_dict['border'].append(border)
            fielddf_dict['aver_rate'].append(aver_rate)
            fielddf_dict['peak_rate'].append(peak_rate)
            fielddf_dict['rate_angle'].append(rate_angle)
            fielddf_dict['rate_R'].append(rate_R)
            fielddf_dict['rate_R_pval'].append(rate_R_pval)
            fielddf_dict['precess_df'].append(fitted_precessdf)
            fielddf_dict['precess_angle'].append(precess_angle)
            fielddf_dict['precess_angle_low'].append(precess_angle_low)
            fielddf_dict['precess_R'].append(precess_R)
            fielddf_dict['precess_R_pval'].append(precess_R_pval)
            fielddf_dict['numpass_at_precess'].append(numpass_at_precess)
            fielddf_dict['numpass_at_precess_low'].append(numpass_at_precess_low)


fielddf_raw = pd.DataFrame(fielddf_dict)

# Assign field ids within ca
fielddf_raw['fieldid_ca'] = 0
for ca, cadf in fielddf_raw.groupby('ca'):
    fieldidca = np.arange(cadf.shape[0]) + 1
    index_ca = cadf.index
    fielddf_raw.loc[index_ca, 'fieldid_ca'] = fieldidca

# fielddf_raw.to_pickle(save_pth)

1/47 trial, CA1, 0/21 field
1/47 trial, CA1, 1/21 field
1/47 trial, CA1, 2/21 field
1/47 trial, CA1, 3/21 field
1/47 trial, CA1, 4/21 field
1/47 trial, CA1, 5/21 field
1/47 trial, CA1, 6/21 field
1/47 trial, CA1, 7/21 field
1/47 trial, CA1, 8/21 field
1/47 trial, CA1, 9/21 field
1/47 trial, CA1, 10/21 field
1/47 trial, CA1, 11/21 field
1/47 trial, CA1, 12/21 field
1/47 trial, CA1, 13/21 field
1/47 trial, CA1, 14/21 field
1/47 trial, CA1, 15/21 field
1/47 trial, CA1, 16/21 field
1/47 trial, CA1, 17/21 field
1/47 trial, CA1, 18/21 field
1/47 trial, CA1, 19/21 field
1/47 trial, CA1, 20/21 field
1/47 trial, CA3, 0/25 field
1/47 trial, CA3, 1/25 field
1/47 trial, CA3, 2/25 field
1/47 trial, CA3, 3/25 field
1/47 trial, CA3, 4/25 field
1/47 trial, CA3, 5/25 field
1/47 trial, CA3, 6/25 field
1/47 trial, CA3, 7/25 field
1/47 trial, CA3, 8/25 field
1/47 trial, CA3, 9/25 field
1/47 trial, CA3, 10/25 field
1/47 trial, CA3, 11/25 field
1/47 trial, CA3, 12/25 field
1/47 trial, CA3, 13/25 field
1/47 

/home/yiu/projects/hippocampus/common/comput_utils.py:1372: UserWarning: Shuffling window is smaller than the spike train! Using window size = range/5 instead.
  warnings.warn("Shuffling window is smaller than the spike train! Using window size = range/5 instead.")
/home/yiu/projects/hippocampus/common/comput_utils.py:1217: RuntimeWarning: Mean of empty slice
  errd = np.nanmean(directionality - dtmp_norm) ** 2
/home/yiu/projects/hippocampus/common/comput_utils.py:1218: RuntimeWarning: Mean of empty slice
  errp = np.nanmean(positionality - ptmp_norm) ** 2


1/47 trial, CA3, 17/25 field
1/47 trial, CA3, 18/25 field
1/47 trial, CA3, 19/25 field
1/47 trial, CA3, 20/25 field
1/47 trial, CA3, 21/25 field
1/47 trial, CA3, 22/25 field
1/47 trial, CA3, 23/25 field
1/47 trial, CA3, 24/25 field
2/47 trial, CA1, 0/35 field
2/47 trial, CA1, 1/35 field
2/47 trial, CA1, 2/35 field
2/47 trial, CA1, 3/35 field
2/47 trial, CA1, 4/35 field
2/47 trial, CA1, 5/35 field
2/47 trial, CA1, 6/35 field
2/47 trial, CA1, 7/35 field
2/47 trial, CA1, 8/35 field
2/47 trial, CA1, 9/35 field
2/47 trial, CA1, 10/35 field
2/47 trial, CA1, 11/35 field
2/47 trial, CA1, 12/35 field
2/47 trial, CA1, 13/35 field
2/47 trial, CA1, 14/35 field
2/47 trial, CA1, 15/35 field
2/47 trial, CA1, 16/35 field
2/47 trial, CA1, 17/35 field
2/47 trial, CA1, 18/35 field
2/47 trial, CA1, 19/35 field
2/47 trial, CA1, 20/35 field


/home/yiu/projects/hippocampus/common/comput_utils.py:1372: UserWarning: Shuffling window is smaller than the spike train! Using window size = range/5 instead.
  warnings.warn("Shuffling window is smaller than the spike train! Using window size = range/5 instead.")


2/47 trial, CA1, 21/35 field
2/47 trial, CA1, 22/35 field
2/47 trial, CA1, 23/35 field
2/47 trial, CA1, 24/35 field
2/47 trial, CA1, 25/35 field
2/47 trial, CA1, 26/35 field
2/47 trial, CA1, 27/35 field


...

44/47 trial, CA3, 5/32 field
44/47 trial, CA3, 6/32 field
44/47 trial, CA3, 7/32 field
44/47 trial, CA3, 8/32 field
44/47 trial, CA3, 9/32 field
44/47 trial, CA3, 10/32 field
44/47 trial, CA3, 11/32 field
44/47 trial, CA3, 12/32 field
44/47 trial, CA3, 13/32 field
44/47 trial, CA3, 14/32 field
44/47 trial, CA3, 15/32 field
44/47 trial, CA3, 16/32 field
44/47 trial, CA3, 17/32 field
44/47 trial, CA3, 18/32 field


/home/yiu/projects/hippocampus/common/comput_utils.py:1372: UserWarning: Shuffling window is smaller than the spike train! Using window size = range/5 instead.
  warnings.warn("Shuffling window is smaller than the spike train! Using window size = range/5 instead.")
/home/yiu/projects/hippocampus/common/comput_utils.py:1217: RuntimeWarning: Mean of empty slice
  errd = np.nanmean(directionality - dtmp_norm) ** 2
/home/yiu/projects/hippocampus/common/comput_utils.py:1218: RuntimeWarning: Mean of empty slice
  errp = np.nanmean(positionality - ptmp_norm) ** 2


44/47 trial, CA3, 19/32 field
44/47 trial, CA3, 20/32 field
44/47 trial, CA3, 21/32 field
44/47 trial, CA3, 22/32 field
44/47 trial, CA3, 23/32 field
44/47 trial, CA3, 24/32 field
44/47 trial, CA3, 25/32 field
44/47 trial, CA3, 26/32 field


/home/yiu/projects/hippocampus/common/comput_utils.py:1372: UserWarning: Shuffling window is smaller than the spike train! Using window size = range/5 instead.
  warnings.warn("Shuffling window is smaller than the spike train! Using window size = range/5 instead.")


44/47 trial, CA3, 27/32 field
44/47 trial, CA3, 28/32 field
44/47 trial, CA3, 29/32 field
44/47 trial, CA3, 30/32 field
44/47 trial, CA3, 31/32 field
46/47 trial, CA3, 0/28 field
46/47 trial, CA3, 1/28 field
46/47 trial, CA3, 2/28 field
46/47 trial, CA3, 3/28 field
46/47 trial, CA3, 4/28 field
46/47 trial, CA3, 5/28 field
46/47 trial, CA3, 6/28 field
46/47 trial, CA3, 7/28 field
46/47 trial, CA3, 8/28 field
46/47 trial, CA3, 9/28 field
46/47 trial, CA3, 10/28 field
46/47 trial, CA3, 11/28 field
46/47 trial, CA3, 12/28 field
46/47 trial, CA3, 13/28 field
46/47 trial, CA3, 14/28 field
46/47 trial, CA3, 15/28 field
46/47 trial, CA3, 16/28 field
46/47 trial, CA3, 17/28 field
46/47 trial, CA3, 18/28 field
46/47 trial, CA3, 19/28 field
46/47 trial, CA3, 20/28 field
46/47 trial, CA3, 21/28 field
46/47 trial, CA3, 22/28 field
46/47 trial, CA3, 23/28 field
46/47 trial, CA3, 24/28 field
46/47 trial, CA3, 25/28 field
46/47 trial, CA3, 26/28 field
46/47 trial, CA3, 27/28 field


In [57]:
fielddf_raw.to_pickle('data/crcns-hc-3/hc3processed_combinedSessions.pickle')